In [18]:
import numpy as np
from fvgp.ensemble_gp import EnsembleGP
from fvgp.ensemble_gp import hyperparameters

In [19]:
def func(x):
    i = np.where(x[:,0] < 1.0)
    r = np.zeros((len(x[:,0])))
    r[i] = 1.0
    return r

def mean1(obj,x,hyperparameters):
    return np.ones(len(x)) * hps[-1]
def mean2(obj,x,hyperparameters):
    return np.ones(len(x)) * hps[-1]
def kernel1(x1,x2,hps,obj):
    d = abs(np.subtract.outer(x1[:,0],x2[:,0]))
    return hps[0] * obj.exponential_kernel(d,hps[1])
def kernel2(x1,x2,hps,obj):
    d = abs(np.subtract.outer(x1[:,0],x2[:,0]))
    return hps[0] * obj.exponential_kernel(d,hps[1])

In [20]:
weights = np.ones(2) / 2.0
weights_bounds = np.array([[0.01,1],[0.01,1]])
hps = [np.ones(3)*5.0,np.ones([3])]
hps_bounds = [np.array([[0.01,100],[0.01,100],[0.0,0.001]]),np.array([[0.01,100],[0.01,100],[1.0,1.001]])]
hps_obj = hyperparameters(weights,weights_bounds,hps,hps_bounds)

a = hps_obj.vectorize_hps(weights,hps)
print(a)
a = hps_obj.vectorize_hps(weights,hps)
print("---------")
print(hps_obj.devectorize_hps(a))
print("bounds unvectorized:")
print(weights_bounds)
print(hps_bounds)
print("bounds vectorized")
b = hps_obj.vectorize_bounds(weights_bounds,hps_bounds)
print(b)
print("bounds unvectorized again")
w,h = hps_obj.devectorize_bounds(b)
print(w)
print(h)

[0.5 0.5 5.  5.  5.  1.  1.  1. ]
---------
(array([0.5, 0.5]), [array([5., 5., 5.]), array([1., 1., 1.])])
bounds unvectorized:
[[0.01 1.  ]
 [0.01 1.  ]]
[array([[1.e-02, 1.e+02],
       [1.e-02, 1.e+02],
       [0.e+00, 1.e-03]]), array([[1.000e-02, 1.000e+02],
       [1.000e-02, 1.000e+02],
       [1.000e+00, 1.001e+00]])]
bounds vectorized
[[1.000e-02 1.000e+00]
 [1.000e-02 1.000e+00]
 [1.000e-02 1.000e+02]
 [1.000e-02 1.000e+02]
 [0.000e+00 1.000e-03]
 [1.000e-02 1.000e+02]
 [1.000e-02 1.000e+02]
 [1.000e+00 1.001e+00]]
bounds unvectorized again
[[0.01 1.  ]
 [0.01 1.  ]]
[array([[1.e-02, 1.e+02],
       [1.e-02, 1.e+02],
       [0.e+00, 1.e-03]]), array([[1.000e-02, 1.000e+02],
       [1.000e-02, 1.000e+02],
       [1.000e+00, 1.001e+00]])]


In [21]:
N = 20
points = np.empty((N,1))
points = np.random.uniform(low = -3, high = 3, size = points.shape)
values = func(points)
print("Base GP successfully initiated")
values = np.empty((N,2))
values[:,0] = func(points)
values[:,1] = func(points) + 10.0

print(hps_obj.hps)

my_gp = EnsembleGP(1,points,values,2,hps_obj,
                gp_kernel_functions = [kernel1,kernel2],
                gp_mean_functions = [mean1,mean2],
                compute_device = "cpu")
my_gp.train(hps_obj,
                pop_size = 20,
                tolerance = 0.0001,
                max_iter = 20)

Base GP successfully initiated
[array([5., 5., 5.]), array([1., 1., 1.])]
CAUTION: you have not provided data variances in fvGP,
they will be set to 1 percent of the data values!


ValueError: operands could not be broadcast together with shapes (20,) (3,) 

In [5]:
import matplotlib.pyplot as plt
x_input = np.empty((1000,1))
x_input[:,0] = np.linspace(-3,3,1000)
y = func(x_input)
pred1_mean = my_gp.posterior_mean(x_input)
pred1_cov = my_gp.posterior_covariance(x_input)

plt.figure(figsize = (10,4))
plt.plot(x_input,pred1_mean["f(x)"], label = "posterior mean",linewidth = 3.0)
plt.plot(x_input,y, label = "ground truth",linewidth = 3.0)
m = pred1_mean["f(x)"]
s = np.diag(pred1_cov["S(x)"])
plt.plot(x_input, s, label = "std", linewidth = 3.0)
plt.fill_between(x_input[:,0], m-3.0*np.sqrt(s), m+3.0*np.sqrt(s), alpha = 0.5, label = "95% confidence interval")
plt.scatter(my_gp.data_x[:,0],my_gp.data_y[0:len(x_input)], label = "data",linewidth = 3.0)
plt.legend()
print("computing probability of the given values")
comp_mean_vec = np.array([2.0,1.0])
comp_var = np.zeros((2, 2))
np.fill_diagonal(comp_var,np.random.rand(len(comp_var)))
x_input_prob = np.array([[0.55],[1.4]])
print("mean: ",comp_mean_vec)
print("var: ",comp_var)
print("points: ", x_input_prob)

AttributeError: 'EnsembleGP' object has no attribute 'posterior_mean'